# Recommandation

Nous voulons recommander des séries à nos utilisateurs. Pour ce faire, nous voulons utiliser deux méthodes (et les comparer).  
En premier, nous utilisons le filtrage collaboratif pour prédire les notes qu'un utilisateur va mettre à une série, puis nous recommandons les séries les mieux notées.  
Ensuite, nous utilisons le contenu des séries et leur proximité (sur les sous-titres) pour prédire les notes qu'un utilisateur va mettre à une série, puis nous recommandons les mieux notées.  
Nous utilisons enfin la nDCG comme mesure d'évaluation de nos recommandations.

In [1]:
from utils.collaborative import *
from utils.similarities import *
from utils.predictions_content import *
from utils.ndcg import *
import operator
import pickle
from collections import OrderedDict
from utils.predictions_notes import *
from utils.recommandation import *

Import collaborative.py ok
Import similarities ok
Import predictions_notes ok
Import predictions_content ok
Import recommandation ok
Import ndcg ok


In [2]:
#path_d_user = "/Users/constancescherer/Desktop/pickles/d_user.p"
#path_sim = "/Users/constancescherer/Desktop/pickles/sim.p"
#path_most_sim = "/Users/constancescherer/Desktop/pickles/most_sim.p"
#path_d_pert_user = "/Users/constancescherer/Desktop/pickles/d_pert_user_k3.p"

path_d_user = "/Vrac/PLDAC_addic7ed/pickles/d_user.p"
path_sim = "/Vrac/PLDAC_addic7ed/pickles/sim.p"
path_most_sim = "/Vrac/PLDAC_addic7ed/pickles/most_sim.p"
path_d_pert_user = "/Vrac/PLDAC_addic7ed/pickles/d_pert_user_k3.p"


# dictionnaire d_users
# {username : {serie : rating}}
with open(path_d_user, 'rb') as pickle_file:
    d_user = pickle.load(pickle_file)

# matrice des similarités cosinus
with open(path_sim, 'rb') as pickle_file:
    sim = pickle.load(pickle_file)

# dictionnaire des séries les plus similaires
with open(path_most_sim, 'rb') as pickle_file:
    most_similar = pickle.load(pickle_file)
    
with open(path_d_pert_user, 'rb') as pickle_file:
    d_pert_user = pickle.load(pickle_file)


In [3]:
#path_ratings = "/Users/constancescherer/Desktop/ratings/ratings_imdb/users"
path_ratings = "/Vrac/PLDAC_addic7ed/ratings/ratings_imdb/users"

liste_series = get_liste_series(d_user)
data = get_data(d_user)
all_data, num_user, num_item = get_all_data(data)
train, train_mat, test = get_train_test(num_user, num_item, all_data, test_size=10)
mean, u_means, i_means,U_ksvd, I_ksvd =  get_Uksvd_Iksvd(train, train_mat, num_user, num_item)
d_username_id, d_itemname_id, Full = create_sparse_mat(data)


#path_series = "/Users/constancescherer/Desktop/addic7ed_final"
path_series = '/Vrac/PLDAC_addic7ed/addic7ed_final'

d_info, d_name = getDicts(path_series)
d_ind = reverse_dict(d_name)
d_titre_filename = get_d_titre_filename("titles/title-filename.txt")
d_filename_titre = reverse_dict(d_titre_filename)
d_id_username = reverse_dict(d_username_id)
d_id_serie = reverse_dict(d_itemname_id)

reversed_u_dic, reversed_i_dic = create_reversed_dic(d_username_id, d_itemname_id)

In [6]:
len(os.dir("path_series"))

AttributeError: module 'os' has no attribute 'dir'

## Filtrage collaboratif

Pour le filtrage collaboratif, nous utilisons une fonction de prédiction des notes basées sur le kSVD (la NMF ?) pour prédire les notes d'un utilisateur.  
Cette fonction prend en compte les biais utilisateur et item dans le calcul de la note.  
Les notes supérieures à 10 sont remises à 10 et les notes inférieures à 1 sont remises à 1.  
On calcule la moyenne de la nDCG sur tous les utilisateurs.

In [4]:
ndcg_moy_fc(d_pert_user, d_username_id, 
        d_itemname_id, 
        d_user,
        U_ksvd,
        I_ksvd,
        u_means,
        i_means,
        mean)

0.045093703623889446

## Contenu

Pour la recommandation par le contenu, on prédit la note d'une série en faisant la moyenne des notes que l'utilisateur a données aux kppv de la série.  
On calcule la moyenne de la ndcg sur tous les utilisateurs pour plusieurs corpus créés avec différents paramètres.

### Premier essai

In [5]:
ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.03370791848374078

### Corpus voc = 25000 ; min_df = 20 ; max_df = 0.4

In [6]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_25000_mindf_20_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_25000_mindf_20_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.008462550798484904

### Corpus voc = 50000 ; min_df = 20 ; max_df = 0.4

In [7]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_50000_mindf_20_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_50000_mindf_20_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.00858315350303621

### Corpus voc = 53540 ; min_df = 20 ; max_df = 0.4

In [8]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_53540_mindf_20_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_53540_mindf_20_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.00859205905799698

### Corpus voc = 141418 ; min_df = 3 ; max_df = 0.4

In [9]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_141418_mindf_3_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_141418_mindf_3_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.009343573118280614

### Corpus voc = 50000 ; min_df = 3 ; max_df = 0.4

In [10]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_50000_mindf_3_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_50000_mindf_3_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.009453971314197306

### Corpus voc = 50000 ; min_df = 10 ; max_df = 0.4

In [11]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_50000_mindf_10_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_50000_mindf_10_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)

ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.009190918384756074

### Corpus voc = 50000 ; min_df = 15 ; max_df = 0.4

In [12]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_50000_mindf_15_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_50000_mindf_15_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)
ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.008659011747069902

### Corpus voc = 50000 ; min_df = 5 ; max_df = 0.4

In [13]:
path_mat_corpus = "/Users/constancescherer/Desktop/pickles_new/pickles/mat_voc_50000_mindf_5_max_df_0.4.p"
#path_mat_corpus = "/Vrac/PLDAC_addic7ed/pickles_new/pickles/mat_voc_50000_mindf_5_max_df_0.4.p"
with open(path_mat_corpus, 'rb') as pickle_file:
    mat_corpus = pickle.load(pickle_file)
    
sim = similarities_from_sparse_mat(mat_corpus)
ndcg_moy_content(d_pert_user,
        d_username_id,
        d_itemname_id,
        d_name,
        d_user,
        d_ind, 
        d_titre_filename, 
        d_filename_titre, 
        d_id_username, 
        d_id_serie, 
        sim )

0.00921873772152227

Les résultats sont assez bas, mais c'est sûrement du au fait que les jugements de pertinence sont très bas aussi. On va essayer de changer les jugements de pertinence de la façon suivante : dans la matrice de pertinence, si le jugement est différent de 0, on le remplace par un 1, sinon on le laisse à 0. On n'a plus des jugements plus ou moins bons, juste si des séries sont pertinentes ou non pour d'autres séries. Ensuite, on refait les pertinences utilisateurs avec ça et on voit si ça change quelque chose.  
Pour l'instant, c'est la recommandation basée sur le contenu qui fonctionne le mieux, avec le corpus formé avec un vocabulaire de 50 000 mots, un min_df de 3 et un max_df de 0.4 qui a le meilleur score.  
Il reste toujours le problème que des séries très peu notées arrivent toujours en tête, surtout pour le filtrage collaboratif (hypothèse : leur biais item est très élevé car le peu de gens les ayants notées les ont très bien notées). Pour palier ce problème, nous proposons l'idée suivante : parmi les séries ayant la même note prédite pour un utilisateur, nous les classons par popularité (nombre de notes supérieures à 7). La majorité des utilisateurs mettant de très bonnes notes aux séries qu'ils ont notées, leur biais utilisateur est également grand, et on se retrouve avec des utilisateurs qui ont beaucoup de séries avec des notes prédites de 10. Parmi celles-ci, il nous semble plus pertinent de recommander en premier les séries les plus populaires.

**<font color='red'>À faire**

- Pertinences séries {0, 1}
- Pertinences utilisateurs qui en découlent
- Tri par popularité à l'intérieur des recommandations